In [1]:
import os, sys, importlib

import geopandas as gpd
import networkx as nx

# Your GOSTnets path could be different
sys.path.append(r"C:\Work\Code\GOSTnets")
import GOSTnets as gn

In [2]:
# Read in input shapefile and convert to G object
input_shapefile = "cxb_roads_edit.shp"

inD = gpd.read_file(input_shapefile)
G = nx.from_pandas_edgelist(inD, edge_attr=['length', 'infra_type', "osm_id", 'geometry'], source="stnode", target="endnode")
G = G.to_directed()

In [3]:
# inspect the resulting Graph
nodes = list(G.nodes(data=True))
edges = list(G.edges(data=True))
print(len(nodes))
print(nodes[0])
print(len(edges))
print(edges[0])

7922
(0, {})
18047
(0, 4690, {'length': 0.128829058995489, 'infra_type': 'unclassified', 'osm_id': '644427970', 'geometry': <shapely.geometry.linestring.LineString object at 0x0000021E94EBBC88>})


In [4]:
# Identify only the largest graph
list_of_subgraphs = list(nx.strongly_connected_component_subgraphs(G))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i

In [6]:
# Write the largest graph back to file
all_data = {}
idx=0
for o, d, data in G.edges(data=True):
    data['o'] = o
    data['d'] = d
    all_data[idx] = data    
    idx = idx+1
big_graph = gpd.GeoDataFrame(all_data).transpose()
big_graph.to_csv(input_shapefile.replace(".shp", "_big_boy.csv"))
big_graph.head()